# כיוונון עדין של מודלים של Open AI

מחברת זו מבוססת על ההנחיות העדכניות שמופיעות בתיעוד [Fine Tuning](https://platform.openai.com/docs/guides/fine-tuning?WT.mc_id=academic-105485-koreyst) של Open AI.

כיוונון עדין משפר את הביצועים של מודלים בסיסיים עבור היישום שלך על ידי אימון נוסף עם נתונים והקשר רלוונטיים למקרה השימוש הספציפי. שים לב שטכניקות כמו _few shot learning_ ו-_retrieval augmented generation_ מאפשרות לך להעשיר את הפרומפט בנתונים רלוונטיים כדי לשפר את האיכות. עם זאת, שיטות אלו מוגבלות על ידי גודל חלון הטוקנים המרבי של המודל הבסיסי.

בכיוונון עדין, אנחנו למעשה מאמנים מחדש את המודל עצמו עם הנתונים הנדרשים (מה שמאפשר לנו להשתמש בהרבה יותר דוגמאות ממה שנכנס בחלון הטוקנים המרבי) – ומפרסים גרסה _מותאמת אישית_ של המודל, שכבר לא צריכה לקבל דוגמאות בזמן ההסקה. זה לא רק משפר את היעילות של עיצוב הפרומפט שלנו (יש לנו יותר גמישות להשתמש בחלון הטוקנים לדברים אחרים), אלא גם עשוי להוזיל עלויות (על ידי הקטנת מספר הטוקנים שצריך לשלוח למודל בזמן ההסקה).

לכיוונון עדין יש 4 שלבים:
1. להכין את נתוני האימון ולהעלות אותם.
1. להריץ את משימת האימון כדי לקבל מודל מכוונן.
1. להעריך את המודל המכוונן ולשפר את האיכות במידת הצורך.
1. לפרוס את המודל המכוונן להסקה כאשר מרוצים מהתוצאה.

שים לב שלא כל המודלים הבסיסיים תומכים בכיוונון עדין – [בדוק בתיעוד של OpenAI](https://platform.openai.com/docs/guides/fine-tuning/what-models-can-be-fine-tuned?WT.mc_id=academic-105485-koreyst) את המידע העדכני ביותר. ניתן גם לכוונן מודל שכבר עבר כיוונון עדין בעבר. במדריך זה נשתמש ב-`gpt-35-turbo` כמודל הבסיסי שלנו לכיוונון עדין.

---


### שלב 1.1: הכנת מערך הנתונים שלך

בואו נבנה צ'אטבוט שיעזור לך להבין את הטבלה המחזורית של היסודות על ידי מענה לשאלות על יסוד כלשהו באמצעות לימריק. במדריך הפשוט _הזה_, ניצור מערך נתונים לאימון המודל עם כמה דוגמאות תגובה שמדגימות את הפורמט המצופה של הנתונים. במקרה אמיתי, תצטרך ליצור מערך נתונים עם הרבה יותר דוגמאות. ייתכן גם שתוכל להשתמש במערך נתונים פתוח (בתחום היישום שלך) אם קיים כזה, ולעצב אותו מחדש כך שיתאים לכיול עדין.

מכיוון שאנו מתמקדים ב-`gpt-35-turbo` ומעוניינים בתגובה בודדת (השלמת צ'אט), נוכל ליצור דוגמאות באמצעות [הפורמט המומלץ הזה](https://platform.openai.com/docs/guides/fine-tuning/preparing-your-dataset?WT.mc_id=academic-105485-koreyst) שמשקף את הדרישות של השלמת צ'אט ב-OpenAI. אם אתה מצפה לתוכן שיחה רב-שלבי, תשתמש ב[פורמט הדוגמה הרב-שלבי](https://platform.openai.com/docs/guides/fine-tuning/multi-turn-chat-examples?WT.mc_id=academic-105485-koreyst) שכולל פרמטר `weight` שמסמן אילו הודעות יש להשתמש בהן (או לא) בתהליך הכיול העדין.

במדריך שלנו נשתמש בפורמט הפשוט של תגובה בודדת. הנתונים נמצאים בפורמט [jsonl](https://jsonlines.org/?WT.mc_id=academic-105485-koreyst) עם רשומה אחת בכל שורה, כאשר כל אחת מיוצגת כאובייקט בפורמט JSON. הקטע למטה מציג 2 רשומות כדוגמה – ראה [training-data.jsonl](../../../../../18-fine-tuning/python/openai/training-data.jsonl) עבור מערך הדוגמאות המלא (10 דוגמאות) שנשתמש בו במדריך הכיול העדין שלנו. **לתשומת לבך:** כל רשומה _חייבת_ להיות מוגדרת בשורה אחת (ולא מפוצלת למספר שורות כפי שנהוג בקובץ JSON מעוצב)

```json
{ "messages": [{"role": "system", "content": "Elle is a factual chatbot that answers questions about elements in the periodic table with a limerick"}, {"role": "user", "content": "Tell me about Gallium"}, {"role": "assistant", "content": "Gallium, oh gallium, so light - Melts in your hand, oh what a sight - At 86 degrees - Its liquid with ease - And in semiconductors, it's out of sight"}]}
{ "messages": [{"role": "system", "content": "Elle is a factual chatbot that answers questions about elements in the periodic table with a limerick"}, {"role": "user", "content": "Tell me about Hydrogen"}, {"role": "assistant", "content": "Hydrogen, the first in the line - The lightest of all, so divine - It's in water, you see - And in stars, it's the key - The universe's most common sign"}]}
```

במקרה אמיתי תצטרך מערך דוגמאות גדול בהרבה כדי לקבל תוצאות טובות – יש כאן איזון בין איכות התגובות לבין הזמן/העלות של הכיול העדין. אנחנו משתמשים במערך קטן כדי שנוכל להשלים את הכיול במהירות ולהדגים את התהליך. ראה [דוגמה זו מה-OpenAI Cookbook](https://github.com/openai/openai-cookbook/blob/main/examples/How_to_finetune_chat_models.ipynb?WT.mc_id=academic-105485-koreyst) למדריך הכיול העדין המורכב יותר.


### שלב 1.2 העלאת מערך הנתונים שלך

העלה את הנתונים באמצעות Files API [כמתואר כאן](https://platform.openai.com/docs/guides/fine-tuning/upload-a-training-file). שים לב שכדי להריץ את הקוד הזה, עליך לבצע קודם את השלבים הבאים:
 - התקנת חבילת ה-Python `openai` (ודא שאתה משתמש בגרסה >=0.28.0 כדי לקבל את כל הפיצ'רים העדכניים)
 - הגדרת משתנה הסביבה `OPENAI_API_KEY` עם מפתח ה-API שלך ל-OpenAI
למידע נוסף, עיין ב[מדריך ההגדרה](./../../../00-course-setup/02-setup-local.md?WT.mc_id=academic-105485-koreyst) שמסופק עבור הקורס.

כעת, הרץ את הקוד כדי ליצור קובץ להעלאה מתוך קובץ ה-JSONL המקומי שלך.


In [24]:
from openai import OpenAI
client = OpenAI()

ft_file = client.files.create(
  file=open("./training-data.jsonl", "rb"),
  purpose="fine-tune"
)

print(ft_file)
print("Training File ID: " + ft_file.id)

FileObject(id='file-JdAJcagdOTG6ACNlFWzuzmyV', bytes=4021, created_at=1715566183, filename='training-data.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)
Training File ID: file-JdAJcagdOTG6ACNlFWzuzmyV


### שלב 2.1: יצירת משימת כיוונון עדין בעזרת ה-SDK


In [25]:
from openai import OpenAI
client = OpenAI()

ft_filejob = client.fine_tuning.jobs.create(
  training_file=ft_file.id, 
  model="gpt-3.5-turbo"
)

print(ft_filejob)
print("Fine-tuning Job ID: " + ft_filejob.id)

FineTuningJob(id='ftjob-Usfb9RjasncaZ5Cjbuh1XSCh', created_at=1715566184, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-EZ6ag0n0S6Zm8eV9BSWKmE6l', result_files=[], seed=830529052, status='validating_files', trained_tokens=None, training_file='file-JdAJcagdOTG6ACNlFWzuzmyV', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None)
Fine-tuning Job ID: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh


### שלב 2.2: בדיקת סטטוס המשימה

הנה כמה דברים שאפשר לעשות עם ה-API של `client.fine_tuning.jobs`:
- `client.fine_tuning.jobs.list(limit=<n>)` - הצגת n משימות הכוונון האחרונות
- `client.fine_tuning.jobs.retrieve(<job_id>)` - קבלת פרטים על משימת כוונון מסוימת
- `client.fine_tuning.jobs.cancel(<job_id>)` - ביטול משימת כוונון
- `client.fine_tuning.jobs.list_events(fine_tuning_job_id=<job_id>, limit=<b>)` - הצגת עד n אירועים מהמשימה
- `client.fine_tuning.jobs.create(model="gpt-35-turbo", training_file="your-training-file.jsonl", ...)`

השלב הראשון בתהליך הוא _אימות קובץ האימון_ כדי לוודא שהנתונים בפורמט הנכון.


In [26]:
from openai import OpenAI
client = OpenAI()

# List 10 fine-tuning jobs
client.fine_tuning.jobs.list(limit=10)

# Retrieve the state of a fine-tune
client.fine_tuning.jobs.retrieve(ft_filejob.id)

# List up to 10 events from a fine-tuning job
client.fine_tuning.jobs.list_events(fine_tuning_job_id=ft_filejob.id, limit=10)

SyncCursorPage[FineTuningJobEvent](data=[FineTuningJobEvent(id='ftevent-GkWiDgZmOsuv4q5cSTEGscY6', created_at=1715566184, level='info', message='Validating training file: file-JdAJcagdOTG6ACNlFWzuzmyV', object='fine_tuning.job.event', data={}, type='message'), FineTuningJobEvent(id='ftevent-3899xdVTO3LN7Q7LkKLMJUnb', created_at=1715566184, level='info', message='Created fine-tuning job: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh', object='fine_tuning.job.event', data={}, type='message')], object='list', has_more=False)

In [30]:
# Once the training data is validated
# Track the job status to see if it is running and when it is complete
from openai import OpenAI
client = OpenAI()

response = client.fine_tuning.jobs.retrieve(ft_filejob.id)

print("Job ID:", response.id)
print("Status:", response.status)
print("Trained Tokens:", response.trained_tokens)

Job ID: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh
Status: running
Trained Tokens: None


### שלב 2.3: עקבו אחרי אירועים כדי לעקוב אחרי ההתקדמות


In [44]:
# You can also track progress in a more granular way by checking for events
# Refresh this code till you get the `The job has successfully completed` message
response = client.fine_tuning.jobs.list_events(ft_filejob.id)

events = response.data
events.reverse()

for event in events:
    print(event.message)

Step 85/100: training loss=0.14
Step 86/100: training loss=0.00
Step 87/100: training loss=0.00
Step 88/100: training loss=0.07
Step 89/100: training loss=0.00
Step 90/100: training loss=0.00
Step 91/100: training loss=0.00
Step 92/100: training loss=0.00
Step 93/100: training loss=0.00
Step 94/100: training loss=0.00
Step 95/100: training loss=0.08
Step 96/100: training loss=0.05
Step 97/100: training loss=0.00
Step 98/100: training loss=0.00
Step 99/100: training loss=0.00
Step 100/100: training loss=0.00
Checkpoint created at step 80 with Snapshot ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWyyF2:ckpt-step-80
Checkpoint created at step 90 with Snapshot ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWyzhK:ckpt-step-90
New fine-tuned model created: ft:gpt-3.5-turbo-0125:bitnbot::9OFWzNjz
The job has successfully completed


### שלב 2.4: הצג סטטוס בלוח הבקרה של OpenAI


ניתן גם לצפות בסטטוס על ידי ביקור באתר של OpenAI ובחירה בסעיף _Fine-tuning_ בפלטפורמה. שם תוכלו לראות את הסטטוס של המשימה הנוכחית, וגם לעקוב אחרי היסטוריית הרצות קודמות. בצילום המסך הזה, אפשר לראות שההרצה הקודמת נכשלה, והריצה השנייה הצליחה. לשם הבהרה, זה קרה כאשר ההרצה הראשונה השתמשה בקובץ JSON עם רשומות בפורמט שגוי – לאחר שתוקן, ההרצה השנייה הושלמה בהצלחה והמודל הפך לזמין לשימוש.

![Fine-tuning job status](../../../../../translated_images/fine-tuned-model-status.563271727bf7bfba7e3f73a201f8712fae3cea1c08f7c7f12ca469c06d234122.he.png)


ניתן גם לצפות בהודעות הסטטוס והמדדים על ידי גלילה מטה בלוח הבקרה הוויזואלי כפי שמוצג:

| הודעות | מדדים |
|:---|:---|
| ![Messages](../../../../../translated_images/fine-tuned-messages-panel.4ed0c2da5ea1313b3a706a66f66bf5007c379cd9219cfb74cb30c0b04b90c4c8.he.png) |  ![Metrics](../../../../../translated_images/fine-tuned-metrics-panel.700d7e4995a652299584ab181536a6cfb67691a897a518b6c7a2aa0a17f1a30d.he.png)|


### שלב 3.1: שליפת מזהה ובדיקת המודל המותאם בקוד



In [46]:
# Retrieve the identity of the fine-tuned model once ready
response = client.fine_tuning.jobs.retrieve(ft_filejob.id)
fine_tuned_model_id = response.fine_tuned_model
print("Fine-tuned Model ID:", fine_tuned_model_id)

Fine-tuned Model ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWzNjz


In [47]:
# You can then use that model to generate completions from the SDK as shown
# Or you can load that model into the OpenAI Playground (in the UI) to validate it from there.
from openai import OpenAI
client = OpenAI()

completion = client.chat.completions.create(
  model=fine_tuned_model_id,
  messages=[
    {"role": "system", "content": "You are Elle, a factual chatbot that answers questions about elements in the periodic table with a limerick"},
    {"role": "user", "content": "Tell me about Strontium"},
  ]
)
print(completion.choices[0].message)

ChatCompletionMessage(content="Strontium, a metal so bright - It's in fireworks, a dazzling sight - It's in bones, you see - And in tea, it's the key - It's the fortieth, so pure, that's the right", role='assistant', function_call=None, tool_calls=None)


### שלב 3.2: טעינה ובדיקה של המודל המותאם ב-Playground

עכשיו אפשר לבדוק את המודל המותאם בשתי דרכים. ראשית, אפשר להיכנס ל-Playground ולהשתמש בתפריט המודלים כדי לבחור את המודל המותאם החדש מתוך האפשרויות שמופיעות. אפשרות נוספת היא להשתמש באופציית "Playground" שמופיעה בלוח הבקרה של ההתאמה (ראו צילום מסך למעלה), שמציגה תצוגה _השוואתית_ שבה רואים את גרסת הבסיס ואת הגרסה המותאמת זו לצד זו, כדי להעריך במהירות את ההבדלים.

![Fine-tuning job status](../../../../../translated_images/fine-tuned-playground-compare.56e06f0ad8922016497d39ced3d84ea296eec89073503f2bf346ec9718f913b5.he.png)

פשוט מלאו את ההקשר של המערכת ששימש אתכם בנתוני האימון והכניסו את שאלת הבדיקה שלכם. תראו ששני הצדדים מתעדכנים עם אותו הקשר ואותה שאלה. הריצו את ההשוואה ותוכלו לראות את ההבדלים בתשובות בין המודלים. _שימו לב איך המודל המותאם מחזיר תשובה בפורמט שסיפקתם בדוגמאות, בעוד שמודל הבסיס פשוט עוקב אחרי ההנחיה של המערכת_.

![Fine-tuning job status](../../../../../translated_images/fine-tuned-playground-launch.5a26495c983c6350c227e05700a47a89002d132949a56fa4ff37f266ebe997b2.he.png)

תשימו לב שגם מוצגים מספר הטוקנים של כל מודל והזמן שלקח לבצע את החיזוי. **הדוגמה הספציפית הזו היא פשוטה ונועדה להמחיש את התהליך, אך לא משקפת נתוני אמת או תרחיש אמיתי**. ייתכן שתשימו לב שבשני המקרים מוצג אותו מספר טוקנים (כי ההקשר והשאלה זהים), אך למודל המותאם לוקח יותר זמן לבצע את החיזוי (מודל מותאם אישית).

בתרחישים אמיתיים, לא תשתמשו בדוגמה פשוטה כזו, אלא תבצעו התאמה לנתונים אמיתיים (למשל, קטלוג מוצרים לשירות לקוחות), שם איכות התשובה תהיה הרבה יותר ברורה. בהקשר כזה, כדי לקבל איכות תשובה דומה ממודל הבסיס, תצטרכו להשקיע יותר בהנדסת פרומפטים, מה שיגדיל את השימוש בטוקנים ואולי גם את זמן העיבוד של החיזוי. _כדי להתנסות בזה, מומלץ לעיין בדוגמאות ההתאמה האישית ב-OpenAI Cookbook ולהתחיל משם._



---

Certainly! Here is your text translated into Hebrew:

**כתב ויתור**:  
מסמך זה תורגם באמצעות שירות תרגום מבוסס בינה מלאכותית [Co-op Translator](https://github.com/Azure/co-op-translator). למרות שאנו שואפים לדיוק, יש להיות מודעים לכך שתרגומים אוטומטיים עלולים להכיל שגיאות או אי-דיוקים. המסמך המקורי בשפת המקור ייחשב כמקור הסמכותי. למידע קריטי, מומלץ לפנות לתרגום מקצועי על ידי אדם. איננו אחראים לכל אי-הבנה או פרשנות שגויה הנובעת מהשימוש בתרגום זה.
